In [ ]:
from numpy import asarray
import numpy as np
import h5py
import os
import cv2
# insert the path of the training folder
cwd_images = './training-testing'
# insert the path of the labels folder
cwd_labels = './labels'
# insert the path of the folder for the training images normalized
train_folder = './data/Synapse/train_npz/case0'
# insert the path of the folder for the testing images normalized
test_folder = './data/Synapse/test_vol_h5/case0' 


for filename in zip(os.listdir(cwd_images),os.listdir(cwd_labels)):
    uniqueName = filename[0][19:-4]
    img = cv2.imread(cwd_images + '\\' + filename[0])
    label = cv2.imread(cwd_labels + '\\' + filename[1])
    
    #Convert them to numpy format, 
    data = asarray(img)
    label_data = asarray(label)
  
    #clip the images within [-125, 275], 
    data_clipped = np.clip(data, -125, 275)

    #normalize each 3D image to [0, 1], and 
    data_normalised = (data_clipped - (-125)) / (275 - (-125))
    
    
    print(data_normalised.shape)
    for i in range(data_normalised.shape[2]):
        formattedi = "{:03d}".format(i)
       
        slice000 = data_normalised[:,:,i]
        label_slice000 = label_data[:,:,i]
        np.savez(train_folder + uniqueName +  '_slice'+formattedi+".npz",
                  image=slice000,label=label_slice000)
    #keep the 3D volume in h5 format for testing cases.
    fn = test_folder + uniqueName + '.npy.h5'
    f = h5py.File(fn, 'w')
    dset = f.create_dataset("image", data=data_normalised)
    dset = f.create_dataset("label", data=label_data)
    f.close()